# Hamming weight extractor

## Hamming weight of the state after the SBox in the first round (first state byte).

### Append a hamming weight column to the ASCAD file.

In [1]:
import h5py
from src.aes.main import AES
from src.data.loaders.ascad import ASCADData, ASCAD_DATA, ASCAD_DATA_VAR, ASCADDataType, ASCAD_DATA_FIX
from tqdm import tqdm
import numpy as np

ascad = ASCADData.fixed_key()
plaintext_ix = 0
full_key_ix = 1

pm = ascad["Profiling_traces"]["metadata"]
am = ascad["Attack_traces"]["metadata"]

def get_acc(metadata):
    return np.zeros((len(metadata), 16), dtype=np.uint8)

profile_key, profile_plain = get_acc(pm), get_acc(pm)
attack_key, attack_plain = get_acc(am), get_acc(am)

for ix in tqdm(range(len(pm))):
    profile_key[ix] = pm[ix][full_key_ix]
    profile_plain[ix] = pm[ix][plaintext_ix]

for ix in tqdm(range(len(am))):
    attack_key[ix] = am[ix][full_key_ix]
    attack_plain[ix] = pm[ix][plaintext_ix]

ascad.close()

100%|██████████| 10000/10000 [00:02<00:00, 3786.04it/s]


In [2]:
from src.aes.hw_sbox import hw_sbox_round1

def gen_hws(plaintext, key):
    """
    Extracts the hamming weight of the first state byte, after the SBox of the first round of AES.
    """
    hws = np.zeros((len(plaintext), 10, 16), dtype=np.uint8)

    # noinspection PyShadowingNames
    for ix in tqdm(range(len(plaintext))):
        aes = AES(key[ix])
        aes.encrypt(plaintext[ix])

        hws[ix] = aes.hw_after_round

    return hws

profile_hw = gen_hws(profile_plain, profile_key)
attack_hw = gen_hws(attack_plain, attack_key)

100%|██████████| 10000/10000 [00:03<00:00, 2886.09it/s]


In [3]:
np.mean(profile_hw), np.mean(attack_hw)

(4.000020875, 4.000711875)

In [4]:
wrt = h5py.File(f"{ASCAD_DATA}{ASCAD_DATA_FIX}/{ASCADDataType.default}.h5", 'r+')

for grp_name, grp_data in [("Profiling_traces", profile_hw), ("Attack_traces", attack_hw)]:
    grp = wrt[grp_name]

    if "hamming_weights" in grp:
        del grp["hamming_weights"]
    grp["hamming_weights"] = grp_data

wrt.close()

chk = h5py.File(f"{ASCAD_DATA}{ASCAD_DATA_FIX}/{ASCADDataType.default}.h5", 'r')
print(list(chk["Profiling_traces"]))
print(list(chk["Attack_traces"]))
chk.close()

['hamming_weights', 'labels', 'metadata', 'traces']
['hamming_weights', 'labels', 'metadata', 'traces']


In [5]:
ascad = ASCADData.fixed_key()
ascad["Profiling_traces"]["hamming_weights"][0]

array([[5, 5, 3, 6, 7, 2, 6, 6, 3, 1, 5, 5, 3, 5, 3, 5],
       [4, 5, 3, 5, 3, 5, 6, 6, 4, 3, 5, 4, 4, 4, 4, 2],
       [5, 3, 4, 4, 5, 2, 5, 5, 6, 7, 4, 4, 4, 3, 4, 4],
       [4, 4, 2, 3, 4, 5, 3, 2, 5, 6, 5, 7, 5, 4, 5, 2],
       [4, 6, 3, 5, 3, 6, 3, 4, 5, 6, 6, 4, 2, 5, 4, 1],
       [5, 3, 4, 5, 6, 3, 5, 3, 5, 2, 6, 1, 5, 4, 3, 6],
       [6, 3, 4, 4, 4, 5, 5, 2, 5, 5, 4, 4, 3, 6, 4, 3],
       [3, 4, 3, 4, 3, 4, 6, 5, 3, 4, 3, 7, 2, 4, 6, 2],
       [4, 6, 2, 4, 2, 3, 6, 5, 2, 5, 6, 4, 5, 0, 7, 4],
       [4, 4, 4, 3, 4, 5, 3, 5, 4, 1, 5, 5, 2, 5, 4, 4]], dtype=uint8)